In [15]:
import cv2
import numpy as np

VIDEO = "Traffic_Laramie_2.mp4"

# ---- Tunables ----
ROI_FRAC = (0.45, 0.90, 0.10, 0.90)  # (y1, y2, x1, x2) as fractions of H/W
COUNT_LINE_FRAC = 0.55               # y position of count line inside ROI (0=top of ROI, 1=bottom)
N_BASELINE = 90
BLUR_K = 21
DIFF_THRESH = 30
MIN_AREA = 1700
ALPHA_SMOOTH = 0.92
MAX_MATCH_DIST = 60
MAX_MATCH_DIST_STATE1 = 140 
AGE_MAX = 10
AGE_MAX_STATE1 = 30 
WRITE_OUT = True
# ---- New tunables for the L->R then up path ----
VLINE_FRAC = 0.45       # vertical line inside ROI: x = frac*ROI_width
HLINE_FRAC = 0.55       # horizontal line inside ROI: y = frac*ROI_height
ORDER_WINDOW = 290       # max frames allowed between first and second crossing
MIN_SPEED = 0.4         # px/frame to ignore jitter

# Matching / lifecycle
IOU_MIN = 0.10                # first try to match by IoU >= this
DIST_MAX = 80                 # px fallback distance gate (state=0/2)
DIST_MAX_STATE1 = 160         # bigger gate during turn (state=1)
MISS_MAX = 8                  # frames you can miss before deleting track
MISS_MAX_STATE1 = 24          # keep state=1 tracks longer

# ---- Helpers ----

def bbox_from_cxcywh(cx, cy, w, h):
    x1 = cx - w/2; y1 = cy - h/2
    x2 = x1 + w;  y2 = y1 + h
    return (x1, y1, x2, y2)

def iou(boxA, boxB):
    ax1, ay1, ax2, ay2 = boxA
    bx1, by1, bx2, by2 = boxB
    inter_x1 = max(ax1, bx1); inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2); inter_y2 = min(ay2, by2)
    iw = max(0.0, inter_x2 - inter_x1); ih = max(0.0, inter_y2 - inter_y1)
    inter = iw * ih
    if inter <= 0: return 0.0
    areaA = (ax2-ax1)*(ay2-ay1); areaB = (bx2-bx1)*(by2-by1)
    return inter / (areaA + areaB - inter + 1e-6)

def extract_roi(frame):
    h, w = frame.shape[:2]
    y1, y2, x1, x2 = ROI_FRAC
    Y1, Y2, X1, X2 = int(y1*h), int(y2*h), int(x1*w), int(x2*w)
    return frame[Y1:Y2, X1:X2], (Y1, X1)

def to_gray_blur(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.GaussianBlur(g, (BLUR_K, BLUR_K), 0)

# ---- Build baseline (median of first N frames) ----
cap = cv2.VideoCapture(VIDEO)
if not cap.isOpened():
    raise RuntimeError(f"Could not open {VIDEO}")

gray_stack = []
while len(gray_stack) < N_BASELINE:
    ok, f = cap.read()
    if not ok:
        break
    roi, _ = extract_roi(f)
    gray_stack.append(to_gray_blur(roi))
if not gray_stack:
    raise RuntimeError("No frames read to build baseline.")
baseline = np.median(np.stack(gray_stack, axis=0), axis=0).astype(np.uint8)

# Reset to start
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
fps = cap.get(cv2.CAP_PROP_FPS) or 25
w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

if WRITE_OUT:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(VIDEO.replace(".mp4","_counted.mp4"), fourcc, fps, (w, h))

# ---- Tracking + Counting ----
tracks = []  # each: {'cx','cy','w','h','age','counted','last_cy'}
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
total_count = 0
frame_idx = 0
next_id = 0 

while True:
    ok, frame = cap.read()
    if not ok:
        break
    frame_idx += 1

    roi, (offY, offX) = extract_roi(frame)
    g = to_gray_blur(roi)

    # foreground mask (replace your block)
    delta = cv2.absdiff(g, baseline)
    _, mask = cv2.threshold(delta, DIFF_THRESH, 255, cv2.THRESH_BINARY)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)  # <-- NEW
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)


    # detections
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    curr = []
    for c in cnts:
        if cv2.contourArea(c) < MIN_AREA:
            continue
        x, y, w2, h2 = cv2.boundingRect(c)
        cx, cy = x + w2/2, y + h2/2
        curr.append([cx, cy, w2, h2])

    # Build detection list with boxes (x1,y1,x2,y2) for IoU
    dets = []
    for c in cnts:
        if cv2.contourArea(c) < MIN_AREA: continue
        x, y, w2, h2 = cv2.boundingRect(c)
        cx, cy = x + w2/2, y + h2/2
        dets.append({'cx':cx,'cy':cy,'w':w2,'h':h2,'box':(x, y, x+w2, y+h2)})
    
    used_det = [False]*len(dets)

    # build proposals (ti, di, iou)
    proposals = []
    for ti, tr in enumerate(tracks):
        tbox = bbox_from_cxcywh(tr['cx'], tr['cy'], tr['w'], tr['h'])
        best = (-1, 0.0)
        for di, d in enumerate(dets):
            if used_det[di]: continue
            j = iou(tbox, d['box'])
            if j > best[1]: best = (di, j)
        if best[0] != -1 and best[1] >= IOU_MIN:
            proposals.append((ti, best[0], best[1], 1 if tr.get('state',0)==1 else 0, tr.get('age',0)))
    
    # commit: armed first, then higher IoU, then older track
    proposals.sort(key=lambda x: (x[3], x[2], -x[4]), reverse=True)
    
    matched_tracks = {}
    matched_dets = set()
    for ti, di, _, _, _ in proposals:
        if ti in matched_tracks or di in matched_dets: continue
        matched_tracks[ti] = di
        matched_dets.add(di); used_det[di] = True

    for ti, di in matched_tracks.items():
        tr = tracks[ti]
        d  = dets[di]
        a = ALPHA_SMOOTH
        tr['cx'] = a*tr['cx'] + (1-a)*d['cx']
        tr['cy'] = a*tr['cy'] + (1-a)*d['cy']
        tr['w']  = a*tr['w']  + (1-a)*d['w']
        tr['h']  = a*tr['h']  + (1-a)*d['h']
        tr['age'] = 0
        tr['misses'] = 0

    # so the distance-fallback can skip matched tracks:
    matched_t = set(matched_tracks.keys())
    
    # 2) Fallback distance matching for still-unmatched tracks/detections
    for ti, tr in enumerate(tracks):
        if ti in matched_t: continue
        gate = DIST_MAX_STATE1 if tr.get('state',0)==1 else DIST_MAX
        best, bestd = -1, 1e18
        for di, d in enumerate(dets):
            if used_det[di]: continue
            dx = tr['cx'] - d['cx']; dy = tr['cy'] - d['cy']
            dist2 = dx*dx + dy*dy
            if dist2 < bestd:
                bestd, best = dist2, di

        if best != -1 and bestd <= gate*gate:
            d = dets[best]
            used_det[best] = True
            a = ALPHA_SMOOTH
            tr['cx'] = a*tr['cx'] + (1-a)*d['cx']
            tr['cy'] = a*tr['cy'] + (1-a)*d['cy']
            tr['w']  = a*tr['w']  + (1-a)*d['w']
            tr['h']  = a*tr['h']  + (1-a)*d['h']
            tr['age'] = 0
            tr['misses'] = 0
        else:
            tr['age'] += 1
            tr['misses'] += 1
    
    # 3) Spawn tracks for any leftover detections
    for di, d in enumerate(dets):
        if used_det[di]: continue
        duplicate = False
        for tr in tracks:
            thr = 0.55 if tr.get('state',0)==1 else 0.35
            if iou(bbox_from_cxcywh(tr['cx'],tr['cy'],tr['w'],tr['h']), d['box']) > thr:
                duplicate = True; break
            dx, dy = tr['cx'] - d['cx'], tr['cy'] - d['cy']
            r2 = 50*50 if tr.get('state',0)==1 else 90*90
            if dx*dx + dy*dy < r2:
                duplicate = True; break
        if duplicate:
            continue
            
        tracks.append({
            'id': next_id, 'cx': d['cx'], 'cy': d['cy'], 'w': d['w'], 'h': d['h'],
            'age': 0, 'misses': 0, 'counted': False,
            'last_cx': d['cx'], 'last_cy': d['cy'], 'state': 0, 't1': -1
        })
        next_id += 1
    
    # 4) Prune with state-aware miss limits (keep turn-state longer)
    pruned = []
    for t in tracks:
        lim = MISS_MAX_STATE1 if t.get('state',0)==1 else MISS_MAX
        if t['misses'] < lim:
            pruned.append(t)
    tracks = pruned

    # ---- L->R then UP crossing logic + overlay ----
    roi_h, roi_w = roi.shape[0], roi.shape[1]
    vx = int(VLINE_FRAC * roi_w)
    hy = int(HLINE_FRAC * roi_h)
    
    # ensure debug-safe defaults
    for tr in tracks:
        tr.setdefault('last_cx', tr['cx'])
        tr.setdefault('last_cy', tr['cy'])
        tr.setdefault('state', 0)   # 0: none, 1: passed vertical L->R, 2: counted
        tr.setdefault('t1', -1)
        tr.setdefault('counted', False)
        tr.setdefault('id', id(tr)) # quick unique-ish label
    
    for tr in tracks:
        px, py = tr['last_cx'], tr['last_cy']
        cx, cy = tr['cx'], tr['cy']
    
        # crossed if prev and curr are on opposite sides
        vx_cross = (px - vx) * (cx - vx) < 0
        hy_cross = (py - hy) * (cy - hy) < 0
    
        sx = cx - px
        sy = cy - py
    
        # Step 1: must cross vertical line going L->R first
        if tr['state'] == 0 and vx_cross and sx > MIN_SPEED:
            tr['state'] = 1
            tr['t1'] = frame_idx
    
        # Step 2: within window, cross horizontal line going UP
        if tr['state'] == 1:
            if frame_idx - tr['t1'] > ORDER_WINDOW:
                tr['state'], tr['t1'] = 0, -1   # timeout: reset
            elif hy_cross and sy < -MIN_SPEED and not tr['counted']:
                total_count += 1
                tr['counted'] = True
                tr['state'] = 2
    
        # remember for next frame
        tr['last_cx'], tr['last_cy'] = cx, cy
    
    # draw guide lines
    cv2.line(roi, (vx, 0), (vx, roi_h-1), (0,165,255), 2)  # vertical (orange)
    cv2.line(roi, (0, hy), (roi_w-1, hy), (0, 0, 255), 2)  # horizontal (red)
    
    # overlay each track's state (green box; orange if state=1; blue dot if counted)
    for t in tracks:
        x = int(t['cx'] - t['w']/2)
        y = int(t['cy'] - t['h']/2)
        color = (0,255,0)
        if t['state'] == 1: color = (0,165,255)
        cv2.rectangle(roi, (x, y), (x + int(t['w']), y + int(t['h'])), color, 2)
        if t['counted']:
            cv2.circle(roi, (int(t['cx']), int(t['cy'])), 4, (255,0,0), -1)
        cv2.putText(roi, f"id{str(t['id'])[-3:]} s{t['state']}",
                    (x, max(0, y-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    
    # ---- draw the two lines (add to your drawing block) ----
    cv2.line(roi, (vx, 0), (vx, roi_h-1), (0, 165, 255), 2)  # vertical: orange
    cv2.line(roi, (0, hy), (roi_w-1, hy), (0, 0, 255), 2)    # horizontal: red

    # paste ROI back
    frame[offY:offY+roi.shape[0], offX:offX+roi.shape[1]] = roi

    # HUD: totals + cars/min
    elapsed_s = frame_idx / fps
    cpm = total_count / (elapsed_s/60) if elapsed_s > 0 else 0.0
    cv2.putText(frame, f"Count: {total_count}", (20, 35),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 2)
    cv2.putText(frame, f"Cars/min: {cpm:.2f}", (20, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 2)

    cv2.imshow("car count", frame)
    if WRITE_OUT: out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
if WRITE_OUT: out.release()
cv2.destroyAllWindows()
print(f"Done. Total cars: {total_count}")


Done. Total cars (direction=down): 0
